# CSc 44800 – Artificial Intelligence, Fall 2025
## Assignment 2: A* Search Algorithm
- **Name:** Sehr Abrar  
- **EMPLID:** 24123078
- **Email:** sabrar000@citymail.cuny.edu
- **Submission Date:** September 25, 2025 (11:59 PM EST)
- **Collaboration:** None

## Part 1: Uploading the file on the Jupyter Notebook

In [20]:
# Step 1: Uploading the RomaniaMap.xlsx file (from local computer)
from google.colab import files

uploaded = files.upload()

Saving RomaniaMap.xlsx to RomaniaMap (1).xlsx


In [21]:
# Step 2: Checking that the file uploaded correctly and can be viewed
import pandas as pd

# get the uploaded file name
filename = list(uploaded.keys())[0]
print("Uploaded file:", filename)

# load the excel file (all sheets)
xls = pd.ExcelFile(filename)
print("\nSheets in the Excel file:", xls.sheet_names)

# loop through each sheet to inspect
for sheet in xls.sheet_names:
    print(f"\n--- Sheet: {sheet} ---")
    df_sheet = pd.read_excel(xls, sheet_name=sheet)

    # show first 5 rows
    print("First 5 rows:")
    print(df_sheet.head())

    # show column names
    print("\nColumns:")
    print(df_sheet.columns)

    # show column types
    print("\nColumn types:")
    print(df_sheet.dtypes)

    # quick description of numeric columns
    print("\nNumeric summary:")
    print(df_sheet.describe())


Uploaded file: RomaniaMap (1).xlsx

Sheets in the Excel file: ['Nodes', 'Edges', 'SLD KM', 'SLD MI']

--- Sheet: Nodes ---
First 5 rows:
  Unnamed: 0 Unnamed: 1    Degrees Unnamed: 3   Radians Unnamed: 5
0      Index       City    Lat (N)   Long (E)   Lat (N)   Long (E)
1          0       Arad     46.175    21.3125  0.805906   0.371973
2          1  Bucharest    44.4325  26.103889  0.775493   0.455599
3          2    Craiova  44.333333  23.816667  0.773763   0.415679
4          3    Drobeta  44.633333      22.65  0.778999   0.395317

Columns:
Index(['Unnamed: 0', 'Unnamed: 1', 'Degrees', 'Unnamed: 3', 'Radians',
       'Unnamed: 5'],
      dtype='object')

Column types:
Unnamed: 0    object
Unnamed: 1    object
Degrees       object
Unnamed: 3    object
Radians       object
Unnamed: 5    object
dtype: object

Numeric summary:
       Unnamed: 0 Unnamed: 1  Degrees Unnamed: 3  Radians Unnamed: 5
count          21         21       21         21       21         21
unique         21        

## Part 2: Build the Graph + Heuristic

In [22]:
# Step 1: Build the graph
edges_df = pd.read_excel(xls, sheet_name='Edges')
edges_df.columns = edges_df.columns.str.strip()  # clean column names by removing any extra spaces

graph = {}
for _, row in edges_df.iterrows():
    # get the city names and distance from the current row
    city_a = row['Name(A)']
    city_b = row['Name(B)']
    distance = row['Distance']

    # add city_a to the graph if it's not already there
    if city_a not in graph:
        graph[city_a] = {}
    # add city_b to the graph if it's not already there
    if city_b not in graph:
        graph[city_b] = {}

    # add the connection both ways (undirected graph)
    graph[city_a][city_b] = distance
    graph[city_b][city_a] = distance

# print a quick example to verify the graph structure
print("Graph built! Example connections from Arad:", graph['Arad'])


Graph built! Example connections from Arad: {'Sibiu': 140, 'Timisoara': 118, 'Zerind': 75}


In [23]:
# Step 2: Build heuristic (straight-line distance to Urziceni)
sld_df = pd.read_excel(xls, sheet_name='SLD MI')

# the first row (index 0) from column 4 onward contains city names
cities = [str(c).strip() for c in sld_df.iloc[0, 4:].tolist()]
# print detected cities to make sure they match the graph
print("Cities detected in SLD MI sheet:", cities)

heuristic = {}
# find the column index for Urziceni in the cities list
urz_index = cities.index('Urziceni')

for i, city in enumerate(cities):
    # distance to Urziceni is in row 3+i (first row of numeric distances + city offset)
    # column 4 + urz_index (adjust for metadata columns)
    urz_distance = float(sld_df.iloc[3 + i, 4 + urz_index])
    heuristic[city] = urz_distance

# print an example heuristic to verify
print("Heuristic built! Distance from Arad to Urziceni:", heuristic['Arad'])

Cities detected in SLD MI sheet: ['Arad', 'Bucharest', 'Craiova', 'Drobeta', 'Eforie', 'Fagaras', 'Giurgiu', 'Hirsova', 'Iasi', 'Lugoj', 'Mehadia', 'Neamt', 'Oradea', 'Pitesti', 'Rimnicu Vilcea', 'Sibiu', 'Timisoara', 'Urziceni', 'Vaslui', 'Zerind']
Heuristic built! Distance from Arad to Urziceni: 277.34288500755906


## Part 3: Implement A* Search

In [27]:
# Step 1: Implementing the A* search algorithm
import heapq  # import heapq for priority queue functionality

def a_star_search(graph, start, goal, heuristic):
    # initialize the open set as a priority queue
    # each element is a tuple: (f = g + h, g = cost so far, current city, path taken)
    open_set = []
    heapq.heappush(open_set, (heuristic[start], 0, start, [start]))  # start node

    visited = set()  # keep track of visited cities to avoid revisiting

    while open_set:
        # pop the city with the smallest f value (priority) from the open set
        f, g, current, path = heapq.heappop(open_set)
        print(f"Visiting: {current}, g={g}, h={heuristic[current]}, f={f}, path={path}")

        # if we have reached the goal, return the path and total cost
        if current == goal:
            print("\nGoal reached!")
            return path, g  # g is the total distance travelled

        # mark the current city as visited
        visited.add(current)

        # explore each neighbor of the current city
        for neighbor, distance in graph[current].items():
            if neighbor in visited:
                continue  # skip if neighbor has already been visited
            g_new = g + distance  # calculate new g value (cost so far + distance to neighbor)
            f_new = g_new + heuristic[neighbor]  # f = g + h
            # add neighbor to the open set with updated f, g, and path
            heapq.heappush(open_set, (f_new, g_new, neighbor, path + [neighbor]))

    # if open set is empty and goal not reached, there is no path
    print("No path found.")
    return None, float('inf')


In [25]:
# Step 2: Run A* search from Zerind to Urziceni
# This will print the visiting order, g, h, f values, and the path step-by-step
path, total_cost = a_star_search(graph, "Zerind", "Urziceni", heuristic)

# print the final shortest path and total distance
print("\nShortest path from Zerind to Urziceni:", path)
print("Total distance:", total_cost)

Visiting: Zerind, g=0, h=280.1546455855485, f=280.1546455855485, path=['Zerind']
Visiting: Oradea, g=71, h=279.32155524276885, f=350.32155524276885, path=['Zerind', 'Oradea']
Visiting: Arad, g=75, h=277.34288500755906, f=352.34288500755906, path=['Zerind', 'Arad']
Visiting: Sibiu, g=215, h=142.19054701825303, f=357.190547018253, path=['Zerind', 'Arad', 'Sibiu']
Visiting: Sibiu, g=222, h=142.19054701825303, f=364.190547018253, path=['Zerind', 'Oradea', 'Sibiu']
Visiting: Rimnicu Vilcea, g=295, h=114.22675670673856, f=409.22675670673857, path=['Zerind', 'Arad', 'Sibiu', 'Rimnicu Vilcea']
Visiting: Rimnicu Vilcea, g=302, h=114.22675670673856, f=416.22675670673857, path=['Zerind', 'Oradea', 'Sibiu', 'Rimnicu Vilcea']
Visiting: Fagaras, g=314, h=112.51389683285834, f=426.51389683285834, path=['Zerind', 'Arad', 'Sibiu', 'Fagaras']
Visiting: Fagaras, g=321, h=112.51389683285834, f=433.51389683285834, path=['Zerind', 'Oradea', 'Sibiu', 'Fagaras']
Visiting: Timisoara, g=193, h=273.0601573239792

## Part 4: Results & Discussion

For this assignment, I implemented the A* search algorithm to find the shortest path between cities in Romania. I first processed the data provided in the `RomaniaMap.xlsx` file. I started by building a graph of the cities using the "Edges" sheet, where each city is a node and each connection is an edge weighted by the distance between the cities. I ensured the graph was undirected, so distances were symmetric between any two connected cities.

Next, I constructed a heuristic using the straight-line distances (SLD) to the goal city, Urziceni, from the "SLD MI" sheet. The heuristic was stored as a dictionary mapping each city to its SLD to Urziceni. This step is crucial for A* because it provides an estimate of the remaining cost from any city to the goal, allowing the algorithm to prioritize nodes that are likely to lead to the shortest path.

Once the graph and heuristic were ready, I implemented the A* search algorithm. I used a priority queue to keep track of cities to visit, sorted by their `f = g + h` value, where `g` is the total distance traveled so far and `h` is the heuristic estimate to the goal. I also maintained a set of visited cities to prevent revisiting nodes and entering cycles.

The algorithm works by repeatedly expanding the city with the lowest `f` value. For each city, I considered all its neighbors, calculated their new `g` and `f` values, and added them to the priority queue if they had not been visited. This process continued until the goal city, Urziceni, was reached. At each step, I printed the city being visited, its current cost `g`, heuristic `h`, total `f`, and the path taken so far to make the search transparent and easy to follow.

Following these steps, the algorithm successfully found the shortest path from Zerind to Urziceni. The final path is `['Zerind', 'Arad', 'Sibiu', 'Rimnicu Vilcea', 'Pitesti', 'Bucharest', 'Urziceni']` with a total distance of 578. This result makes sense because A* balances both the actual distance traveled and the estimated remaining distance, efficiently guiding the search along a path that minimizes the total cost.

Overall, the assignment demonstrated how to transform raw city and distance data into a usable graph and heuristic, and how A* uses these structures to systematically find the optimal path. The process highlighted the importance of carefully building the graph and heuristic, since any errors in these steps could mislead the algorithm and result in incorrect paths or distances.
